# Read a STAC Catalog Using PySTAC

In this tutorial, you will gain an understanding of how to explore a STAC Catalog using [PySTAC](https://pystac.readthedocs.io/en/stable/index.html). You will be able to use these skills to explore any of the [many existing STAC Catalogs](https://stacindex.org/catalogs).

This tutorial as well as the following 3 tutorials were adapted from the [PySTAC site tutorials](https://pystac.readthedocs.io/en/stable/index.html).

Throughout this tutorial, a variety of PySTAC classes, methods, and instances are used to read the STAC Catalog. We encourage you to look at the [PySTAC API Reference](https://pystac.readthedocs.io/en/latest/api.html) while going through this tutorial. 

## Dependencies 
If you need to install pystac, uncomment the line below and run this cell.

In [ ]:
# ! pip install pystac

## STAC Catalogs
[A STAC Catalog](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec) is used to group other STAC objects like items, collections, and/or even other catalogs.

We will be using a small example catalog adapted from the [example Landsat Collection](https://github.com/geotrellis/geotrellis-server/tree/977bad7a64c409341479c281c8c72222008861fd/stac-example/catalog/landsat-stac-collection) in the [GeoTrellis](https://geotrellis.io) repository. All STAC Items and Collections can be found in the [docs/example-catalog](https://github.com/stac-utils/pystac/tree/main/docs/example-catalog) directory of the PySTAC documentation repository; all assets are hosted in the Landsat S3 bucket.

#### Import Packages and Store Data
To begin, import the packages and PySTAC classes that you need to access data and work with STAC Catalogs in Python.

In [ ]:
import json

from pystac import Catalog, get_stac_version
from pystac.extensions.eo import EOExtension
from pystac.extensions.label import LabelExtension

In [ ]:
# Read the example catalog
root_catalog = Catalog.from_file('/public/example-catalog/catalog.json')

## Explore the High-Level Catalog Information

To give us an idea of how the catalog we are working with is organized, let's take a look at all elements of the STAC using the `describe` method.

<div style="background-color:rgba(181, 216, 229, 0.8); text-align:center; vertical-align: top; padding:10px; ml:5px; 0;">
NOTE: Be careful using the `describe` method on large catalogs, as it will walk and print the entire tree of the STAC.
</div>

In [ ]:
root_catalog.describe()

From this output, we can see that the catalog has 1 collection and that this collection has 4 items. 

Now, let's look at the root catalog more in depth.

In [ ]:
# Print some basic metadata from the Catalog
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

<div style="background-color:rgba(181, 216, 229, 0.8); text-align:center; vertical-align: top; padding:10px; ml:5px; 0;">
Note: we do not print the "stac_version" here. PySTAC automatically updates any catalogs to the most recent supported STAC version and will automatically write this to the JSON object during serialization.
</div>  


Let's confirm the latest STAC Specification version supported by PySTAC.

In [ ]:
print(get_stac_version())

With this information, we have an understanding of the layout and general information about the STAC Catalog at hand. Now, let's dive deeper into this catalog. 

## Crawl STAC Child Catalogs and/or Collections

[STAC Collections](https://github.com/radiantearth/stac-spec/tree/master/collection-spec) are used to group related items and provide aggregate or summary metadata for those items.

STAC Catalogs may have many nested layers of catalogs or collections within the top-level collection. Our example catalog only has one collection within the main catalog at [landsat-8-l1/collection.json](/public/example-catalog/landsat-8-l1/collection.json). We can list the collections in a given catalog using the `Catalog.get_collections` method. This method returns an iterable of PySTAC [Collection](https://pystac.readthedocs.io/en/latest/api.html#collection) instances, which we will turn into a `list`.

In [ ]:
collections = list(root_catalog.get_collections())

print(f"Number of collections: {len(collections)}")
print("Collections IDs:")
for collection in collections:
    print(f"- {collection.id}")

Let's grab that collection as a PySTAC [Collection](https://pystac.readthedocs.io/en/latest/api/collection.html#pystac-collection) instance using the `Catalog.get_child` method so we can look at it in more detail. This method gets a child catalog or collection by ID, so we'll use the collection ID that we printed above. Since this method returns `None` if no child exists with the given ID, let's check to make sure we actually got the `Collection`.

In [ ]:
collection = root_catalog.get_child("landsat-8-l1")
if collection is None:
    print("Collection is Empty. Check your downloads and try agian.")
else:
    print("Collection has a root child. You may proceed to the following steps.")

## Crawl the STAC Items

[STAC Items](https://github.com/radiantearth/stac-spec/tree/master/item-spec) are the fundamental building blocks of a STAC Catalog. Each Item represents a single spatiotemporal resource (e.g. a satellite scene).

Both catalogs and collections may have items associated with them. Let's crawl our catalog, starting at the root, to see what Items we have. The `Catalog.get_all_items` method provides a convenient way of recursively listing all Items associated with a catalog and all of its sub-catalogs.

In [ ]:
items = list(root_catalog.get_all_items())

print(f"Number of items: {len(items)}")
for item in items:
    print(f"- {item.id}")

These IDs are not very descriptive; in the next section, we will see how we can access the rich metadata associated with each item.

## Explore STAC Item Metadata

### Item Metadata

Items can have *a lot* of metadata. This can be a bit overwhelming at first, but let's break the metadata fields down into a few categories:

1. Core Item Metadata
2. Common Metadata
3. STAC Extensions

We will walk through each of these metadata categories in the following sections. 

First, let's grab one of the Items using the `Catalog.get_item` method. We will use `recursive=True` to recursively crawl all child catalogs and/or collections to find the item.

In [ ]:
item = root_catalog.get_item("LC80140332018166LGN00", recursive=True)

#### 1. Core Item Metadata

The core item metadata fields include spatiotemporal information and the ID of the collection to which the item belongs. These fields are all at the top level of the item JSON and we can access them through attributes on the [PySTAC Item](https://pystac.readthedocs.io/en/latest/api/collection.html#pystac-collection) instance.

In [ ]:
item.geometry

In [ ]:
item.bbox

In [ ]:
item.datetime

In [ ]:
item.collection_id

We can also access the same information as the cell above by running the `Item.get_collection` method.

In [ ]:
item.get_collection()

####  2. Common Metadata

Certain fields that are commonly used in Items, but may also be found in other objects (e.g. Assets) are defined in the [Common Metadata](https://github.com/radiantearth/stac-spec/blob/v1.0.0/item-spec/common-metadata.md) section of the spec. These include licensing and instrument information, descriptions of datetime ranges, and some other common fields. These properties can be found as attributes of the `Item.common_metadata` property, which is an instance of the `CommonMetadata` class.

In [ ]:
item.common_metadata.instruments

In [ ]:
item.common_metadata.platform

In [ ]:
item.common_metadata.gsd

#### 3. STAC Extensions

[STAC Extensions](https://github.com/radiantearth/stac-spec/tree/master/extensions) are a mechanism for providing additional metadata not covered by the core STAC Spec. We can see which STAC Extensions are implemented by this particular Item by examining the list of extension URIs in the `stac_extensions` field.

In [ ]:
item.stac_extensions

This Item implements the [Electro-Optical](https://github.com/stac-extensions/eo), [View Geometry](https://github.com/stac-extensions/view), and [Projection](https://github.com/stac-extensions/projection) Extensions. 

We can also check if a specific extension is implemented using the `has_extension` method for that extension class.

In [ ]:
EOExtension.has_extension(item)

In [ ]:
LabelExtension.has_extension(item)

We can access fields associated with the extension as attributes on the extension instance. For instance, the ["eo:cloud_cover" field](https://github.com/stac-extensions/eo#item-properties-or-asset-fields) defined in the Electro-Optical Extension can be accessed using the [EOExtension.cloud_cover](https://pystac.readthedocs.io/en/latest/api.html#pystac.extensions.eo.EOExtension.cloud_cover) attribute.

In [ ]:
eo_item_ext = EOExtension.ext(item)
eo_item_ext.cloud_cover

We can also access the cloud cover field directly in the Item properties.

In [ ]:
item.properties['eo:cloud_cover']

## Access STAC Item's Assets

To access the item's assets, we can use the `assets` attribute, which is a dictionary:

In [ ]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

We can use the `to_dict()` method to convert an asset, or any PySTAC object, into a dictionary:

In [ ]:
asset = item.assets['B3']
asset.to_dict()

Here, we use the `eo Extension` to get the band information for the asset:

In [ ]:
eo_asset_ext = EOExtension.ext(asset)
bands = eo_asset_ext.bands
bands

In [ ]:
bands[0].to_dict()

You have now successfully explored the common components of an existing STAC Catalog. To learn how to write your own STAC Catalog, see the [following tutorial](/en/tutorials/2-create-stac-catalog-python/index.html).

#### Join the conversation
If you have any questions, you’re welcome to ask our community on [Gitter](https://app.gitter.im/#/room/#SpatioTemporal-Asset-Catalog_Lobby).